In [ ]:
import os
import json
import pandas as pd

# 🛠️ 어휘 다양성 계산 함수
def calculate_lexical_diversity(text):
    """
    텍스트에서 어휘 다양성을 계산합니다.
    :param text: str, 분석할 텍스트
    :return: 어휘 다양성 점수 (고유 단어 수 / 전체 단어 수)
    """
    if not isinstance(text, str) or len(text.strip()) == 0:
        return 0
    words = text.split()  # 단어 단위로 분리
    unique_words = set(words)  # 고유 단어 집합
    lexical_diversity = len(unique_words) / len(words) if len(words) > 0 else 0
    return lexical_diversity

# 📂 JSON 파일로부터 데이터 로드 (Clickbait & NonClickbait 공통)
def load_data_from_json(file_path, label):
    """
    단일 JSON 파일에서 데이터를 로드합니다.
    :param file_path: JSON 파일 경로
    :param label: 1 (Clickbait) 또는 0 (Non-Clickbait)
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if isinstance(data, list):
            df = pd.DataFrame(data)
        elif isinstance(data, dict):
            df = pd.DataFrame(data.get('articles') or data.get('data') or data)
        else:
            print(f"❌ 알 수 없는 JSON 구조입니다: {type(data)}")
            return pd.DataFrame()

        if 'sourceDataInfo' in df.columns:
            df['title'] = df['sourceDataInfo'].apply(lambda x: x.get('newsTitle', '') if isinstance(x, dict) else '')
            df['content'] = df['sourceDataInfo'].apply(lambda x: x.get('newsContent', '') if isinstance(x, dict) else '')

        df['label'] = label  # Clickbait = 1, NonClickbait = 0
        return df[['title', 'content', 'label']]
    
    except Exception as e:
        print(f"❌ 오류 발생: {file_path} - {e}")
        return pd.DataFrame()

# 📂 데이터 경로 설정 (Clickbait & NonClickbait 단일 JSON 파일)
clickbait_path = r'C:\Ryuna\ClickBait Detecting related data\Training\Labeling\TL_Part1_Clickbait_Merged_SO\merged_clickbait.json'
nonclickbait_path = r'C:\Ryuna\ClickBait Detecting related data\Training\Labeling\TL_Part1_NonClickbait_Merged_SO\merged_file.json'


# 🛠️ 데이터 불러오기
print("📂 Clickbait 데이터 불러오는 중...")
clickbait_df = load_data_from_json(clickbait_path, label=1)  # Clickbait 데이터 로드

print("📂 Non-Clickbait 데이터 불러오는 중...")
nonclickbait_df = load_data_from_json(nonclickbait_path, label=0)  # Non-Clickbait 데이터 로드

# 🛠️ 데이터 병합
df = pd.concat([clickbait_df, nonclickbait_df], ignore_index=True)
print(f"📚 데이터 병합 완료. 데이터 크기: {df.shape}")

# 🛠️ 어휘 다양성 및 추가 피처 계산
print("📊 어휘 다양성 및 추가 피처 계산 중...")

# Title Lexical Diversity
df['title_lexical_diversity'] = df['title'].apply(calculate_lexical_diversity)

# Content Lexical Diversity
df['content_lexical_diversity'] = df['content'].apply(calculate_lexical_diversity)

# Combined Lexical Diversity (Title + Content)
df['combined_text'] = df['title'] + ' ' + df['content']
df['combined_lexical_diversity'] = df['combined_text'].apply(calculate_lexical_diversity)

# Title Length (단어 수)
df['title_length'] = df['title'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

# Content Length (단어 수)
df['content_length'] = df['content'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)

# 🔥 사용하지 않는 열 삭제
df = df[['title_lexical_diversity', 'content_lexical_diversity', 'combined_lexical_diversity', 
         'title_length', 'content_length', 'label']]

# 📘 데이터 저장
output_path = r'D:\머신러닝\csv_files\lexical_diversity_results_combined_naivebayes ver_1215.csv'  # 경로 수정
df.to_csv(output_path, index=False)
print(f"✅ 분석 결과가 '{output_path}'에 저장되었습니다.")
